In [1]:
import requests
import pandas as pd
import folium
from folium.plugins import HeatMap
from bs4 import BeautifulSoup
from opencage.geocoder import OpenCageGeocode

In [2]:
def find_long_lat(county):
    API_KEY = ""
    geocoder = OpenCageGeocode(API_KEY)
    location = county + ", NY"
    result = geocoder.geocode(location)
    arr = [result[0]['geometry']['lat'], result[0]['geometry']['lng']]
    return arr

def create_county_df(county, popu):
    pop = popu
    lat = []
    long = []
    
    for i in county:
        temp = find_long_lat(i)
        lat.append(temp[0])
        long.append(temp[1])

    dict = {'name': county, 'pop': pop, 'lat': lat, 'lon': long}
    df = pd.DataFrame(dict)
    df.to_csv('covid-County.csv', encoding='utf-8', index=False)
    print("===== Successfully write CSV file =====")

In [3]:
page = requests.get("https://www.health.ny.gov/diseases/communicable/coronavirus/")
row_digit = []
row_wrd = []

if page.status_code == 200:
    print("===== Successfully connected =====")
    soup = BeautifulSoup(page.content, 'html.parser')

    result = soup.find(id='case_count_table')
    # print(result.prettify())

    for item in result.findAll('td'):
        if item.getText() == "New York State (Outside of NYC)":
            break

        if item.getText().isdigit():
            row_digit.append(item.getText())
        else:
            row_wrd.append(item.getText())
            
    create_county_df(row_wrd, row_digit)
    print("===== Successfully assigned =====")
    
else:
    print("Unseccessfully connect to the URL")

===== Successfully connected =====
===== Successfully write CSV file =====
===== Successfully assigned =====


In [4]:
map = folium.Map(location=[42.6884183, -77.0790249], tiles = "openstreetmap", control_scale=True, zoom_start=7)

df = pd.read_csv('covid-County.csv')
# stationArr = df[['lat', 'lon']].as_matrix()

# plot heatmap
# map.add_children(plugins.HeatMap(stationArr, radius=15))

stationArr = HeatMap(data=df[['lat', 'lon', 'pop']].groupby(['lat', 'lon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13, color="red").add_to(map)

# map.add_children(plugins.HeatMap(stationArr, radius=15))
map